# CCSD theory for a closed-shell reference

In this notebook we will use wicked to generate and implement equations for the CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers.py`.
In this example, we run a CCSD computation on the H<sub>6</sub> molecule, reading all the relevant information from the file `sr-h2o-cc-pvdz-spinorbital.npy`.

In [29]:
import time
import wicked as w
import numpy as np
from examples_helpers import *

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h2o-cc-pvdz-spinorbital.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [30]:
molecule = "sr-h2o-cc-pvdz-spinorbital"

with open(f"{molecule}.npy", "rb") as f:
    Eref = np.load(f)
    Ecorr_ref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()

invD = compute_inverse_denominators(H, nocc, nvir, 2)

## Compute the MP2 energy

To verify that the Hamiltonian is read correctly, we compute the MP2 correlation energy

In [31]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 * invD["oovv"][i][j][a][b]

print(f"MP2 correlation energy: {Emp2:.12f} Eh")

MP2 correlation energy: -0.214347674279 Eh


## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

In [32]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o"])
Hop = w.utils.gen_op("H", 1, "ov", "ov") + w.utils.gen_op("H", 2, "ov", "ov")
# the similarity-transformed Hamiltonian truncated to the four-nested commutator term
Hbar = w.bch_series(Hop, Top, 4)

Tip: if combinatorically equivalent operators are present, they will not be removed from the expression! Set 'unique' to True to change this behavior.

Tip: if combinatorically equivalent operators are present, they will not be removed from the expression! Set 'unique' to True to change this behavior.

Tip: if combinatorically equivalent operators are present, they will not be removed from the expression! Set 'unique' to True to change this behavior.



In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [33]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [34]:
energy_eq = generate_equation(mbeq, 0, 0)
t1_eq = generate_equation(mbeq, 1, 1)
t2_eq = generate_equation(mbeq, 2, 2)

exec(energy_eq)
exec(t1_eq)
exec(t2_eq)

# show what do these functions look like
print(energy_eq)

def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"],optimize="optimal")
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"],optimize="optimal")
    R += 0.500000000 * np.einsum("abij,jb,ia->",H["vvoo"],T["ov"],T["ov"],optimize="optimal")
    return R


## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [35]:
T = {"ov": np.zeros((nocc, nvir)), "oovv": np.zeros((nocc, nocc, nvir, nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 50

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_0_0(H, T)
    Etot_w = Eref + Ecorr_w
    R["ov"] = evaluate_residual_1_1(H, T)
    Roovv = evaluate_residual_2_2(H, T)
    R["oovv"] = antisymmetrize_residual_2_2(Roovv, nocc, nvir)

    # 2. amplitude update
    update_cc_amplitudes(T, R, invD, 2)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R["ov"]) ** 2 + np.linalg.norm(R["oovv"]) ** 2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}")
    if norm_R < 1.0e-8:
        break

end = time.perf_counter()
t = end - start

print("-" * len(header))
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:+.12e} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)

-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -75.989795787487    +0.000000000000    1.962660e+00
  1    -76.204143461766    -0.214347674279    2.526930e-01
  2    -76.207264973707    -0.217469186220    7.602154e-02
  3    -76.211865730825    -0.222069943338    3.035113e-02
  4    -76.212864167040    -0.223068379554    1.371958e-02
  5    -76.213350138227    -0.223554350740    6.433935e-03
  6    -76.213541063949    -0.223745276462    3.176255e-03
  7    -76.213628949546    -0.223833162059    1.615380e-03
  8    -76.213668687064    -0.223872899577    8.550421e-04
  9    -76.213687566002    -0.223891778515    4.664256e-04
 10    -76.213696641243    -0.223900853756    2.619370e-04
 11    -76.213701122859    -0.223905335373    1.502973e-04
 12    -76.213703376304    -0.223907588817    8.771325e-05
 13    -76.213704534002    -0.223908746515    5.18042

### (Optional) Integral generation
For completeness, we document here how `sr-h2o-cc-pvdz-spinorbital.npy` was generated. You will need to install PySCF to run the following cell. Due to the size, we do not provide the cc-pVTZ integrals in the repository, but you can generate it with the cell below and see the difference to the spin-integrated code in the other notebook.

In [27]:
import pyscf, pyscf.cc, pyscf.mp

mol = pyscf.gto.M(atom="""
O 
H 1 1.1
H 1 1.1 2 104
""", basis='cc-pvdz')

mf = pyscf.scf.RHF(mol)
_ = mf.kernel()
cc = pyscf.cc.CCSD(mf)
_ = cc.kernel()
mp = pyscf.mp.MP2(mf)
_ = mp.kernel()

nocc = mol.nelectron
nvir = (mol.nao*2 - nocc)

eri = pyscf.ao2mo.full(mol.intor('int2e'), mf.mo_coeff)
eri = eri.swapaxes(1, 2)

blocks = get_index_blocks(energy_eq+t1_eq+t2_eq)

V = np.zeros((mol.nao*2, mol.nao*2, mol.nao*2, mol.nao*2))
V[::2,::2,::2,::2] = V[1::2,1::2,1::2,1::2] = eri - eri.swapaxes(2,3) # <aa||aa> and <bb||bb>
V[::2,1::2,::2,1::2] = V[1::2,::2,1::2,::2] = eri # <ab||ab> = <ba||ba> = <ab|ab>
V[::2,1::2,1::2,::2] = V[1::2,::2,::2,1::2] = -eri.swapaxes(2,3) # <ab||ba> = <ba||ab> = -<ab|ab>
F_spatorb = np.diag(mf.mo_energy)
F = np.zeros((mol.nao*2, mol.nao*2))
F[::2,::2] = F[1::2,1::2] = F_spatorb
sl = {'o': slice(0, nocc), 'v': slice(nocc, nocc+nvir)}
H = {}
for block in blocks:
    if len(block) == 2:
        H[block] = F[sl[block[0]], sl[block[1]]]
    else:
        H[block] = V[sl[block[0]], sl[block[1]], sl[block[2]], sl[block[3]]]

with open('sr-h2o-cc-pvdz-spinorbital.npy', 'wb') as f:
    np.save(f, mf.e_tot)
    np.save(f, cc.e_corr)
    np.save(f, (nocc,nvir))
    np.save(f, H, allow_pickle=True)

converged SCF energy = -75.9897957874868
E(CCSD) = -76.2137058001166  E_corr = -0.2239100126297487
E(MP2) = -76.2041434617656  E_corr = -0.214347674278744
E(SCS-MP2) = -76.2010219265649  E_corr = -0.211226139078036
